In [1]:
import re
import nltk

import pandas as pd
import numpy as np

Осуществим предобработку данных с Твиттера, чтобы очищенные данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания. Для работы объединим train_df и test_df.

In [2]:
df_train = pd.read_csv('train_tweets.csv',encoding='utf-8')
df_test = pd.read_csv('test_tweets.csv',encoding='utf-8')

In [3]:
df_train.drop("label",axis=1,inplace=True)

In [4]:
df_train.head()

,id,tweet
0,1,@user when a father is dysfunctional and is s...
1,2,@user @user thanks for #lyft credit i can't us...
2,3,bihday your majesty
3,4,#model i love u take with u all the time in ...
4,5,factsguide: society now #motivation


In [5]:
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [6]:
df_train.shape, df_test.shape

((31962, 2), (17197, 2))

In [7]:
df = pd.concat([df_train,df_test],axis=0,ignore_index=True)

In [8]:
df.shape

(49159, 2)

1. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию:
    • для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
    • для для замены @user на пробел, необходимо использовать re.sub()

In [9]:
# убираем все небуквенные символы
regex = re.compile("@[\w]*")

def words_user(text, regex=regex):
    try:
        res = re.findall(regex, text)
        result = re.sub(res[0],'', text)
        return result
    except:
        return text

In [10]:
print(df.tweet[0])

 @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run


In [11]:
print(words_user(df.tweet[0]))

  when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run


In [12]:
df.tweet = df.tweet.apply(words_user)
df.head()

,id,tweet
0,1,when a father is dysfunctional and is so sel...
1,2,thanks for #lyft credit i can't use cause th...
2,3,bihday your majesty
3,4,#model i love u take with u all the time in ...
4,5,factsguide: society now #motivation


2. Изменим регистр твитов на нижний с помощью .lower().

In [13]:
def words_lower(text):
    try:
        lower_text = text.lower()
        return lower_text
    except:
        return text

In [14]:
print(words_lower(df.tweet[0]))

  when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run


In [15]:
df.tweet = df.tweet.apply(words_lower)
df.head()

,id,tweet
0,1,when a father is dysfunctional and is so sel...
1,2,thanks for #lyft credit i can't use cause th...
2,3,bihday your majesty
3,4,#model i love u take with u all the time in ...
4,5,factsguide: society now #motivation


3. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [16]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have",
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

In [17]:
df.tweet.head()

0      when a father is dysfunctional and is so sel...
1      thanks for #lyft credit i can't use cause th...
2                                  bihday your majesty
3    #model   i love u take with u all the time in ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

In [18]:
def replace_dict(text, dictionary):
    words = [dictionary[word] if word in list(dictionary.keys()) else word for word in text.split()]
    return ' '.join(words)

In [19]:
df.tweet = np.vectorize(replace_dict)(df.tweet, apostrophe_dict)
df.tweet = np.vectorize(replace_dict)(df.tweet, short_word_dict)

    5. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [20]:
emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [21]:
df.tweet = np.vectorize(replace_dict)(df.tweet, emoticon_dict)

In [22]:
df.tweet.head()

0    when a father is dysfunctional and is so selfi...
1    thanks for #lyft credit i cannot use cause the...
2                                  bihday your majesty
3    #model i love you take with you all the time i...
4                  factsguide: society now #motivation
Name: tweet, dtype: object

    6. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.
    7. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.
    8. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.
    9. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

In [23]:
def replace_pattern(pattern, repl, string):
    return re.sub(pattern, repl, string)
df.tweet = np.vectorize(replace_pattern)(r'[^\w\s]', ' ', df.tweet)
df.tweet = np.vectorize(replace_pattern)(r'[^a-zA-Z0-9]', ' ', df.tweet)
df.tweet = np.vectorize(replace_pattern)(r'[^a-zA-Z]', ' ', df.tweet)

In [24]:
df.tweet.head()

0    when a father is dysfunctional and is so selfi...
1    thanks for  lyft credit i cannot use cause the...
2                                  bihday your majesty
3     model i love you take with you all the time i...
4                  factsguide  society now  motivation
Name: tweet, dtype: object

10. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [25]:
from nltk.tokenize import word_tokenize
from nltk import tokenize as tknz

In [26]:
df['tweet_token'] = df.tweet.apply(tknz.word_tokenize)
df['tweet_token'].head()

0    [when, a, father, is, dysfunctional, and, is, ...
1    [thanks, for, lyft, credit, i, can, not, use, ...
2                              [bihday, your, majesty]
3    [model, i, love, you, take, with, you, all, th...
4               [factsguide, society, now, motivation]
Name: tweet_token, dtype: object

11. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [27]:
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [28]:
stopwords.fileids()

['arabic',
 'azerbaijani',
 'basque',
 'bengali',
 'catalan',
 'chinese',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hebrew',
 'hinglish',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

In [29]:
stop_words = set(stopwords.words("english"))

def delete_stopwords(tokens, stop_words=stop_words):
    return [w for w in tokens if w not in stop_words]

df['tweet_token_filtered'] = df.tweet_token.apply(delete_stopwords)
df.tweet_token_filtered

0        [father, dysfunctional, selfish, drags, kids, ...
1        [thanks, lyft, credit, use, cause, offer, whee...
2                                        [bihday, majesty]
3                            [model, love, take, time, ur]
4                        [factsguide, society, motivation]
                               ...                        
49154    [thought, factory, left, right, polarisation, ...
49155    [feeling, like, mermaid, hairflip, neverready,...
49156    [hillary, campaigned, today, ohio, omg, amp, u...
49157    [happy, work, conference, right, mindset, lead...
49158    [song, glad, free, download, shoegaze, newmusi...
Name: tweet_token_filtered, Length: 49159, dtype: object

12. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [30]:
stemmer = nltk.stem.PorterStemmer()

def steming_tokens(tokens):
    return [stemmer.stem(w) for w in tokens]

df['tweet_stemmed'] = df.tweet_token_filtered.apply(steming_tokens)
df.tweet_stemmed

0        [father, dysfunct, selfish, drag, kid, dysfunc...
1        [thank, lyft, credit, use, caus, offer, wheelc...
2                                        [bihday, majesti]
3                            [model, love, take, time, ur]
4                              [factsguid, societi, motiv]
                               ...                        
49154    [thought, factori, left, right, polaris, trump...
49155    [feel, like, mermaid, hairflip, neverreadi, fo...
49156    [hillari, campaign, today, ohio, omg, amp, use...
49157    [happi, work, confer, right, mindset, lead, cu...
49158    [song, glad, free, download, shoegaz, newmus, ...
Name: tweet_stemmed, Length: 49159, dtype: object

13. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [31]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def lemmatized_tokens(tokens):
    return [lemmatizer.lemmatize(w) for w in tokens]

df['tweet_lemmatized'] = df.tweet_token_filtered.apply(lemmatized_tokens)
df.tweet_lemmatized


[nltk_data] Downloading package omw-1.4 to /home/recpi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0        [father, dysfunctional, selfish, drag, kid, dy...
1        [thanks, lyft, credit, use, cause, offer, whee...
2                                        [bihday, majesty]
3                            [model, love, take, time, ur]
4                        [factsguide, society, motivation]
                               ...                        
49154    [thought, factory, left, right, polarisation, ...
49155    [feeling, like, mermaid, hairflip, neverready,...
49156    [hillary, campaigned, today, ohio, omg, amp, u...
49157    [happy, work, conference, right, mindset, lead...
49158    [song, glad, free, download, shoegaze, newmusi...
Name: tweet_lemmatized, Length: 49159, dtype: object

14. Сохраним результат предобработки в pickle-файл.

In [32]:
df.to_pickle('Practice_1.pkl')